In [1]:
import json
import random

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
from matplotlib.widgets import Button
import json as js
from sortedcontainers import SortedSet
from queue import PriorityQueue
from functools import cmp_to_key

TOLERANCE = 0.15

def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))

class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = []
        self.adding_lines = False
        self.added_lines = []
        self.adding_rects = False
        self.added_rects = []

    def set_axes(self, ax):
        self.ax = ax

    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        self.draw(autoscaling = True)

    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        self.draw(autoscaling = True)

    def add_point(self, event):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points.append(PointsCollection([]))

    def add_line(self, event):
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines.append(LinesCollection([]))

    def add_rect(self, event):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()

    def new_rect(self):
        self.added_rects.append(LinesCollection([]))
        self.rect_points = []

    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[-1].add_points([new_point])
            self.draw(autoscaling = False)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling = False)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[-1].add([self.rect_points[-1], new_point])
                self.rect_points.append(new_point)
                self.draw(autoscaling = False)
            elif len(self.rect_points) > 1:
                if dist(self.rect_points[0], new_point) < (np.mean([self.ax.get_xlim(), self.ax.get_ylim()])*TOLERANCE):
                    self.added_rects[-1].add([self.rect_points[-1], self.rect_points[0]])
                    self.new_rect()
                else:
                    self.added_rects[-1].add([self.rect_points[-1], new_point])
                    self.rect_points.append(new_point)
                self.draw(autoscaling = False)

    def draw(self, autoscaling = True):
        if not autoscaling:
            xlim = self.ax.get_xlim()
            ylim = self.ax.get_ylim()
        self.ax.clear()
        for collection in (self.scenes[self.i].points + self.added_points):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs)
        for collection in (self.scenes[self.i].lines + self.added_lines + self.added_rects):
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale(autoscaling)
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
        plt.draw()

In [2]:
class Scene:
    def __init__(self, points=[], lines=[]):
        self.points=points
        self.lines=lines

class PointsCollection:
    def __init__(self, points, **kwargs):
        self.points = points
        self.kwargs = kwargs

    def add_points(self, points):
        self.points = self.points + points

class LinesCollection:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs

    def add(self, line):
        self.lines.append(line)

    def get_collection(self):
        return mcoll.LineCollection(self.lines, **self.kwargs)

class Plot:
    def __init__(self, scenes = [Scene()], points = [], lines = [], json = None):
        if json is None:
            self.scenes = scenes
            if points or lines:
                self.scenes[0].points = points
                self.scenes[0].lines = lines
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]],
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]])
                           for scene in js.loads(json)]

    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.2)
        ax_prev = plt.axes([0.6, 0.05, 0.15, 0.075])
        ax_next = plt.axes([0.76, 0.05, 0.15, 0.075])
        ax_add_point = plt.axes([0.44, 0.05, 0.15, 0.075])
        ax_add_line = plt.axes([0.28, 0.05, 0.15, 0.075])
        ax_add_rect = plt.axes([0.12, 0.05, 0.15, 0.075])
        b_next = Button(ax_next, 'Następny')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Poprzedni')
        b_prev.on_clicked(self.callback.prev)
        b_add_point = Button(ax_add_point, 'Dodaj punkt')
        b_add_point.on_clicked(self.callback.add_point)
        b_add_line = Button(ax_add_line, 'Dodaj linię')
        b_add_line.on_clicked(self.callback.add_line)
        b_add_rect = Button(ax_add_rect, 'Dodaj figurę')
        b_add_rect.on_clicked(self.callback.add_rect)
        return [b_prev, b_next, b_add_point, b_add_line, b_add_rect]

    def add_scene(self, scene):
        self.scenes.append(scene)

    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    def toJson(self):
        return js.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points],
                          "lines":[linesCol.lines for linesCol in scene.lines]}
                         for scene in self.scenes])

    def get_added_points(self):
        if self.callback:
            return self.callback.added_points
        else:
            return None

    def get_added_lines(self):
        if self.callback:
            return self.callback.added_lines
        else:
            return None

    def get_added_figure(self):
        if self.callback:
            return self.callback.added_rects
        else:
            return None

    def get_added_elements(self):
        if self.callback:
            return Scene(self.callback.added_points, self.callback.added_lines+self.callback.added_rects)
        else:
            return None

    def draw(self):
        plt.close()
        fig = plt.figure()
        self.callback = _Button_callback(self.scenes)
        self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on = False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        self.callback.draw()

# Rozwiązanie Lab4

### Funkcja generująca losowe zbiory punktów

In [3]:
from random import uniform as ru
def generate_random_lines(n, a1 = 0, b1 = 10, a2 = 0, b2 = 10):
    lines = []
    for i in range(n):
        lines.append([[ru(a1, b1), ru(a2, b2)], [ru(a1, b1), ru(a2, b2)]])
    normalize(lines)
    return filter_out(lines)


### Funkcja do zadawania zbiorów punktów

In [23]:
%matplotlib notebook

plot1 = Plot()
plot1.draw()

<IPython.core.display.Javascript object>

In [24]:
%matplotlib notebook

plot2 = Plot([plot1.get_added_elements()])
with open("lines/number_of_lines.txt", 'r') as f:
    counter = int(f.readline())
counter += 1
with open("lines/number_of_lines.txt", 'w') as f:
    f.write(str(counter))
with open('lines/line' + str(counter) + '.json', 'w') as file:
    file.write(plot2.toJson())
plot2.draw()

<IPython.core.display.Javascript object>

### Funkcja konwertująca plik json na listę odcinków

In [4]:
def convert_js(f_name):
    fl = open(f_name)
    plot = json.load(fl)
    fl.close()
    lines = plot[0]['lines'][0]
    normalize(lines)
    return filter_out(lines)


### Modyfikacja zbioru tak, aby pierwszy punkt odcinka był tym o mniejszej współrzędnej x

In [5]:
def normalize(lines):
    for i in range(len(lines)):
        if lines[i][0] > lines[i][1]:
            lines[i][0], lines[i][1] = lines[i][1], lines[i][0]


### Przyjęta tolerancja dla zera

In [7]:
tolerance = pow(10, -8)

### Funkcja usuwająca linie niespełniające warunków

In [6]:
def filter_out(lines):
    new_lines = []
    lines.sort(key=lambda x: x[1][0])
    for i in range(len(lines)):
        if abs(lines[i][0][0] - lines[i][0][0]) < tolerance:
            if new_lines and abs(new_lines[-1][1][0] - lines[i][1][0]) < tolerance:
                continue
            else:
                new_lines.append(lines[i])
    return new_lines

### Rysowanie wszystkich zbiorów punktów

In [8]:
def plot_all_lines():
    with open("lines/number_of_lines.txt", 'r') as fl:
        no_of_lines = int(fl.readline())
    scenes = []
    for i in range(1, no_of_lines + 1):
        l = convert_js("lines/line"+str(i)+".json")
        scenes.append(Scene(lines=[LinesCollection(l)]))
    return Plot(scenes=scenes)

In [25]:
%matplotlib notebook

pl = plot_all_lines()
pl.draw()

<IPython.core.display.Javascript object>

### Klasa Linia

In [9]:

class Line:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        self.low_y = min(p1[1], p2[1])
        self.high_y = max(p1[1], p2[1])
        self.a = (p1[1] - p2[1]) / (p1[0] - p2[0])
        self.b = (p2[1]*p1[0] - p1[1]*p2[0]) / (p1[0] - p2[0])

    def find_intersect(self, line2):
        if abs(self.a - line2.a) < tolerance:
            return None
        x = (line2.b - self.b) / (self.a - line2.a)
        y = (self.a*line2.b - line2.a*self.b) / (self.a - line2.a)
        if (self.p1[0] < x < self.p2[0] and line2.p1[0] < x < line2.p2[0] and
            self.low_y < y < self.high_y and line2.low_y < y < line2.high_y):
            return x, y
        return None

    def f(self, x):
        return self.a * x + self.b


### Tworzenie list obiektów klasy linia

In [10]:
def make_line_list(lines):
    line_obj = []
    for p1, p2 in lines:
        line_obj.append(Line(p1, p2))
    return line_obj

### Sprawdzanie istnienia przecięcia

In [11]:
def brute_check(lines):
    points = []
    n = len(lines)
    lin_obj = make_line_list(lines)
    i = 0
    found = False
    while i < n - 1 and not found:
        for j in range(i + 1, n):
            point = lin_obj[i].find_intersect(lin_obj[j])
            if point is not None:
                points.append(point)
                found = True
                break
        i += 1
    return Scene(points=[PointsCollection(points, color='red')], lines=[LinesCollection(lines)]), found


In [12]:
def opt_check(lines):
    def check_intersection(x, y):
        check = lis[x].find_intersect(lis[y])
        if check is not None:
            intersection_point.append(check)
            return True
        return False

    def my_cmp(x, y):
        if x == y:
            return 0
        else:
            return lis[x].f(point) - lis[y].f(point)

    lis = make_line_list(lines)
    events = []
    for i in range(len(lines)):
        events.append((lines[i][0][0], True, i))
        events.append((lines[i][1][0], False, i))
    events.sort()
    intersection_point = []
    state = SortedSet(key=cmp_to_key(my_cmp))
    for point, t, i in events:
        if t:
            state.add(i)
            pos = state.index(i)
            if pos > 0 and check_intersection(i, state[pos - 1]):
                break
            if pos < len(state) - 1 and check_intersection(i, state[pos + 1]):
                break
        else:
            pos = state.index(i)
            if 0 < pos < len(state) - 1 and check_intersection(state[pos+1], state[pos-1]):
                break
            state.remove(i)
    return Scene(points=[PointsCollection(intersection_point, color='green')], lines=[LinesCollection(lines)]), len(intersection_point)


In [13]:
def check_all_plots():
    with open("lines/number_of_lines.txt", 'r') as fl:
        no_of_lines = int(fl.readline())
    scenes = []
    for i in range(1, no_of_lines + 1):
        l = convert_js("lines/line"+str(i)+".json")
        new_scene, found = opt_check(l)
        # new_scene, found = brute_check(l)
        scenes.append(new_scene)
    return Plot(scenes=scenes)

In [ ]:
%matplotlib notebook
pl = check_all_plots()
pl.draw()

### Wizualizacja procesu wykrywania przecięcia

In [14]:
def visualize_check(lines):
    def check_intersection(x, y):
        return lis[x].find_intersect(lis[y])

    def add_to_scenes():
        scenes.append(Scene(points=[PointsCollection(intersection_point[:], color='purple')], lines=[LinesCollection(lines),
        LinesCollection([line_sweep], color='red'), LinesCollection(compared[:], color='blue')]))

    def cmp_two_lines(x, y):
        compared.append(lines[x])
        add_to_scenes()
        compared.append(lines[y])
        add_to_scenes()
        check = check_intersection(x, y)
        compared.pop()
        compared.pop()
        if check is not None:
            intersection_point.append(check)
            add_to_scenes()
            return True
        return False

    def my_cmp(x, y):
        if x == y:
            return 0
        else:
            return lis[x].f(point) - lis[y].f(point)

    lis = make_line_list(lines)
    events = []
    for i in range(len(lines)):
        events.append((lines[i][0][0], True, i))
        events.append((lines[i][1][0], False, i))

    min_bound = min(min(lines[i][0][1], lines[i][1][1]) for i in range(len(lines)))
    max_bound = max(max(lines[i][0][1], lines[i][1][1]) for i in range(len(lines)))
    compared = []
    line_sweep = ()
    scenes = []
    events.sort()
    intersection_point = []
    state = SortedSet(key=cmp_to_key(my_cmp))
    for point, t, i in events:
        line_sweep = ((point, min_bound), (point, max_bound))
        add_to_scenes()
        if t:
            state.add(i)
            pos = state.index(i)
            if pos > 0 and cmp_two_lines(i, state[pos - 1]):
                break
            if pos < len(state) - 1 and cmp_two_lines(i, state[pos + 1]):
                break
        else:
            pos = state.index(i)
            if 0 < pos < len(state) - 1 and cmp_two_lines(state[pos + 1], state[pos - 1]):
                break
            state.remove(i)
    return Plot(scenes=scenes)

In [ ]:
%matplotlib notebook
pl = visualize_check(convert_js("lines/line1.json"))
pl.draw()

### Znajdowanie przecięć (algorytm Brute-force)

In [15]:
def brute_intersect(lines):
    points = []
    n = len(lines)
    lin_obj = make_line_list(lines)
    for i in range(n - 1):
        for j in range(i + 1, n):
            point = lin_obj[i].find_intersect(lin_obj[j])
            if point is not None:
                points.append(point)
    return Scene(points=[PointsCollection(points, color='red')], lines=[LinesCollection(lines)]), len(points)

In [16]:
def find_all_intersections():
    with open("lines/number_of_lines.txt", 'r') as fl:
        no_of_lines = int(fl.readline())
    scenes = []
    for i in range(1, no_of_lines + 1):
        l = convert_js("lines/line"+str(i)+".json")
        new_scene, count = opt_intersect(l)
        # new_scene, count = brute_intersect(l)
        scenes.append(new_scene)
    return Plot(scenes=scenes)

### Znajdowanie przecięć

In [17]:
def opt_intersect(lines):
    def check_intersection(x, y):
        if (x, y) not in detected:
            inter_point = lis[x].find_intersect(lis[y])
            if inter_point is not None:
                intersection_points.append(inter_point)
                if lis[x].f(point) > lis[y].f(point):
                    sweep.put((inter_point[0], 2, x, y))
                else:
                    sweep.put((inter_point[0], 2, y, x))
            detected.add((x, y))
            detected.add((y, x))

    def my_cmp(x, y):
        if x == y:
            return 0
        elif (x, y) == special_inequality:
            return 1
        elif (y, x) == special_inequality:
            return -1
        else:
            return lis[x].f(point) - lis[y].f(point)

    intersection_points = []
    lis = make_line_list(lines)
    sweep = PriorityQueue()
    state = SortedSet(key=cmp_to_key(my_cmp))
    detected = set()
    special_inequality = None
    for i in range(len(lines)):
        sweep.put((lines[i][0][0], 0, i, i))
        sweep.put((lines[i][1][0], 1, i, i))

    while not sweep.empty():
        point, t, i, j = sweep.get()
        if t == 0:
            state.add(i)
            pos = state.index(i)
            if pos > 0:
                check_intersection(i, state[pos - 1])
            if pos < len(state) - 1:
                check_intersection(i, state[pos + 1])
        elif t == 1:
            pos = state.index(i)
            if 0 < pos < len(state) - 1:
                check_intersection(state[pos - 1], state[pos + 1])
            state.remove(i)
        else:
            special_inequality = (i, j)
            pos1 = state.index(i)
            pos2 = state.index(j)
            if pos2 > 0:
                check_intersection(i, state[pos2 - 1])
            if pos1 < len(state) - 1:
                check_intersection(j, state[pos1 + 1])
            state.remove(i)
            state.remove(j)
            special_inequality = (j, i)
            state.add(i)
            state.add(j)

    return Scene(points=[PointsCollection(intersection_points, color='green')], lines=[LinesCollection(lines)]), len(intersection_points)

In [ ]:
%matplotlib notebook

pl = find_all_intersections()
pl.draw()

In [18]:
def test1():
    size = [10, 20, 30, 40, 50, 100]
    scenes = []
    for s in size:
        print("Number of lines: ", s)
        lines = generate_random_lines(s)
        scene1, count1 = opt_intersect(lines)
        print("Opt: ", count1, end = " ")
        scenes.append(scene1)
        scene2, count2 = brute_intersect(lines)
        print("Brute: ", count2)
        print("---------------")
        scenes.append(scene2)
    return Plot(scenes=scenes)


In [ ]:
%matplotlib notebook

pl = test1()
pl.draw()

### Wizualizacja procesu zamiatania

In [19]:
def visualize_line_sweep(lines):
    def add_to_scenes():
        scenes.append(Scene(points=[PointsCollection(intersection_points[:], color='purple')], lines=[LinesCollection(lines),
        LinesCollection([line_sweep] + removed[:], color='red'), LinesCollection(compared[:], color='blue'), LinesCollection(new_added[:], color='green')]))

    def check_intersection(x, y):
        if (x, y) not in detected:
            inter_point = lis[x].find_intersect(lis[y])
            if inter_point is not None:
                intersection_points.append(inter_point)
                int_indexes.append((x, y))
                add_to_scenes()
                if lis[x].f(point) > lis[y].f(point):
                    sweep.put((inter_point[0], 2, x, y))
                else:
                    sweep.put((inter_point[0], 2, y, x))
            detected.add((x, y))
            detected.add((y, x))

    def cmp_two_lines(x, y):
        compared.append(lines[x])
        compared.append(lines[y])
        add_to_scenes()
        check_intersection(x, y)
        compared.pop()
        compared.pop()

    def my_cmp(x, y):
        if x == y:
            return 0
        elif (x, y) == special_inequality:
            return 1
        elif (y, x) == special_inequality:
            return -1
        else:
            return lis[x].f(point) - lis[y].f(point)

    new_added = []
    removed = []
    compared = []
    scenes = []
    intersection_points = []
    int_indexes = []
    lis = make_line_list(lines)
    sweep = PriorityQueue()
    state = SortedSet(key=cmp_to_key(my_cmp))
    detected = set()
    special_inequality = None
    for i in range(len(lines)):
        sweep.put((lines[i][0][0], 0, i, i))
        sweep.put((lines[i][1][0], 1, i, i))

    min_bound = min(min(lines[i][0][1], lines[i][1][1]) for i in range(len(lines)))
    max_bound = max(max(lines[i][0][1], lines[i][1][1]) for i in range(len(lines)))
    while not sweep.empty():
        point, t, i, j = sweep.get()
        line_sweep = ((point, min_bound), (point, max_bound))
        add_to_scenes()
        if t == 0:
            new_added.append(lines[i])
            add_to_scenes()
            new_added.pop()
            state.add(i)
            pos = state.index(i)
            if pos > 0:
                cmp_two_lines(i, state[pos - 1])
            if pos < len(state) - 1:
                cmp_two_lines(i, state[pos + 1])
        elif t == 1:
            removed.append(lines[i])
            add_to_scenes()
            removed.pop()
            pos = state.index(i)
            if 0 < pos < len(state) - 1:
                cmp_two_lines(state[pos - 1], state[pos + 1])
            state.remove(i)
        else:
            special_inequality = (i, j)
            pos1 = state.index(i)
            pos2 = state.index(j)
            if pos2 > 0:
                cmp_two_lines(i, state[pos2 - 1])
            if pos1 < len(state) - 1:
                cmp_two_lines(j, state[pos1 + 1])
            state.remove(i)
            state.remove(j)
            special_inequality = (j, i)
            state.add(i)
            state.add(j)

    add_to_scenes()
    return Plot(scenes=scenes), len(intersection_points), [(intersection_points[i], int_indexes[i][0], int_indexes[i][1]) for i in range(len(int_indexes))]

In [26]:
%matplotlib notebook

pl, count_intersections, inter_points = visualize_line_sweep(convert_js("lines/line1.json"))
print(inter_points)
pl.draw()

[((0.008305863259732869, -0.012859704034026203), 3, 4), ((0.019093081780622655, -0.008616553594230603), 1, 4), ((0.014212297472398418, -0.018783510112023123), 3, 1), ((0.02111023418666054, -0.02570173487544484), 2, 3), ((0.029693150439935284, -0.004447018518388278), 2, 4)]


<IPython.core.display.Javascript object>